In [1]:
import pandas as pd
import lightgbm as lgb
from isk17mssql import PostgreSQLSession
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
session = PostgreSQLSession(
        server="localhost",
        db="zakupki",
        user="postgres",
        password="password",
        port='5432'
)

Connected to
-----------------
Server: localhost
Database: zakupki
User: postgres
-----------------


In [3]:
winners = session.select_statement("select * from public.winners")

In [4]:
tenders = session.select_statement("select * from public.tenders")

In [5]:
customers = session.select_statement("select * from public.customers")

In [6]:
customers.head()

,tender_registration_number,short_name,inn,kpp,ogrn,legal_address
0,32009819068,"АО ""ГАЗПРОМ ГАЗОРАСПРЕДЕЛЕНИЕ МАЙКОП""",0105018196,010501001,1020100707318,"385003, Респ Адыгея, г Майкоп, ул Апшеронская,..."
1,32009819073,"АО ""ГАЗПРОМ ГАЗОРАСПРЕДЕЛЕНИЕ МАЙКОП""",0105018196,010501001,1020100707318,"385003, Респ Адыгея, г Майкоп, ул Апшеронская,..."
2,32009809606,"ФГБОУ ВО ""АГУ""",0105017636,010501001,1020100698750,"385000, РЕСПУБЛИКА АДЫГЕЯ,ГОРОД МАЙКОП,УЛИЦА П..."
3,32009809999,"ФГБОУ ВО ""АГУ""",0105017636,010501001,1020100698750,"385000, РЕСПУБЛИКА АДЫГЕЯ,ГОРОД МАЙКОП,УЛИЦА П..."
4,32009813632,"АО ""ГАЗПРОМ ГАЗОРАСПРЕДЕЛЕНИЕ МАЙКОП""",0105018196,010501001,1020100707318,"385003, Респ Адыгея, г Майкоп, ул Апшеронская,..."


In [7]:
tenders.head()

,registration_number,create_date,name
0,32009819068,2020-12-18T10:46:02,Поставка канцелярских товаров
1,32009819073,2020-12-18T10:46:29,Выполнение работ по разработке грунта механиз...
2,32009809606,2020-12-18T10:27:08,Оказание услуг по круглосуточной физической ох...
3,32009809999,2020-12-18T08:56:39,Оказание услуг по техническому обслуживанию ох...
4,32009813632,2020-12-17T11:13:52,Работа по восстановлению асфальтобетонного пок...


In [8]:
winners.head()

,tender_registration_number,create_date,name,inn,kpp,ogrn
0,31908247804,2019-08-30,"ООО ""Азия Синема-М""",7725276249,380143001,1157746517311
1,31705799518,2018-04-04,"АО ""Газпром газораспределение""",1245787897,124578987,1245787898700
2,31806981734,2018-10-22,"Общество с ограниченной ответственностью ""АКВА...",2312206025,231201001,1132312008927
3,31806337501,2018-03-15,"Общество с ограниченной ответственностью ""Став...",2635207033,263501001,1152651000457
4,31806331930,2018-03-20,"Общество с ограниченной ответственностью ""Спектр""",2311216824,231101001,1162375025295


In [9]:
positive = (
    tenders\
    .merge(winners, left_on="registration_number", right_on="tender_registration_number", suffixes=("_t", "_w"))\
    .filter(items=["name_t", "name_w", "inn"])
)
positive["winner"] = 1

In [10]:
others = (
    winners.loc[~winners["tender_registration_number"].isin(tenders["registration_number"])]\
    .sample(n=len(positive.index))[["name", "inn"]]
)

In [11]:
negative = pd.concat([positive[["name_t"]], others], axis=1).rename(columns={"name": "name_w"})
negative["winner"] = 0

In [12]:
negative.dropna(inplace=True)

In [13]:
positive

,name_t,name_w,inn,winner
0,Открытый аукцион в электронной форме на право ...,Общество с ограниченной ответственностью «Наде...,5642005273,1
1,Поставка канцелярских товаров для Федерального...,ООО ОфисЮгСнаб,2310144704,1
2,Поставка канцелярских товаров для Федерального...,ООО ОфисЮгСнаб,2310144704,1
3,Предоставление услуг по техническому обслужива...,"ООО ""КАРАР""",0105006120,1
4,Отбор организации для выполнения работ по уста...,"ООО ""РОСАВТО""",0917020777,1
...,...,...,...,...
3076,на право заключения договора подряда на выполн...,"ООО ""ГлавКроммерц""",7705920948,1
3077,Выполнение работ по строительству администрати...,"ООО ""РегионЭнергоМонтаж""",2222804503,1
3078,Выполнение работ по строительству администрати...,"ООО ""РегионЭнергоМонтаж""",2222804503,1
3079,Привлечение кредитных ресурсов,"""Газпромбанк"" (Открытое акционерное общество) ...",7744001497,1


In [14]:
negative

,name_t,name_w,inn,winner
0,Открытый аукцион в электронной форме на право ...,"ООО ""Азия Синема-М""",7725276249,0
8,Поставка образцовых мерников,"ОАО ""Адыгеянеруд""",0103003042,0
14,Выбор организации для выполнения работ по подг...,"Закрытое акционерное общество ""Нефрес""",7706148474,0
15,Восстановление дорожного покрытия при реконстр...,"Общество с ограниченной ответственностью ""Юг-С...",2311157030,0
16,Закупка полуфабрикатов для производства протез...,"ООО ""Битум Инвест""",6452120110,0
...,...,...,...,...
3021,поставка сценических костюмов,"ООО ""НЕФТЕПРОДУКТ""",2225142713,0
3052,Поставка технической соли (концентрат минераль...,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ «Эко ...,6164107628,0
3060,Поставка семян,"ООО ""Русский Гриб""",2222848557,0
3066,Поставка дизельного топлива (летнего) для нужд...,"ООО ""ТЕХСНАБ""",5404504041,0


In [15]:
Xy_train = pd.concat([positive, negative], axis=0)

In [16]:
Xy_train.head()

,name_t,name_w,inn,winner
0,Открытый аукцион в электронной форме на право ...,Общество с ограниченной ответственностью «Наде...,5642005273,1
1,Поставка канцелярских товаров для Федерального...,ООО ОфисЮгСнаб,2310144704,1
2,Поставка канцелярских товаров для Федерального...,ООО ОфисЮгСнаб,2310144704,1
3,Предоставление услуг по техническому обслужива...,"ООО ""КАРАР""",0105006120,1
4,Отбор организации для выполнения работ по уста...,"ООО ""РОСАВТО""",0917020777,1


In [17]:
name_t_cv = CountVectorizer(max_features=1000)
name_w_cv = CountVectorizer(max_features=1000)
part1 = name_t_cv.fit_transform(Xy_train["name_t"])
part2 = name_w_cv.fit_transform(Xy_train["name_t"])

In [18]:
class Model:
    @staticmethod
    def train(Xy_train):
        name_t_cv = CountVectorizer(max_features=1000)
        name_w_cv = CountVectorizer(max_features=1000)
        name_t_cv.fit_transform(Xy_train["name_t"])
        name_w_cv.fit_transform(Xy_train["name_w"])
        X_train = pd.DataFrame(cosine_similarity(part1, part2).sum(axis=1), columns=["similarity"])
        X_train["inn"] = Xy_train["inn"].reset_index(drop=True).astype('category')
        y_train = Xy_train["winner"]
        clf = lgb.LGBMClassifier(silent=False)
        clf.fit(X_train, y_train)

In [19]:
Model.train(Xy_train)

[LightGBM] [Info] Number of positive: 3081, number of negative: 282
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000027 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 539
[LightGBM] [Info] Number of data points in the train set: 3363, number of used features: 2
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.916146 -> initscore=2.391102
[LightGBM] [Info] Start training from score 2.391102
